<a href="https://colab.research.google.com/github/felixmadeira/design-interiores/blob/main/design_interiores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q google-generativeai google-cloud-vision googletrans==4.0.0-rc1

import google.generativeai as genai
import os
from google.cloud import vision
from google.colab import files
from io import BytesIO
from googletrans import Translator

# Solicita o upload da chave de API do Google Cloud
files.upload()

# Define a variável de ambiente com o caminho da chave de API Vision
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "design-interiores-3bea3f8f2c1c.json" # Substitua pelo nome do seu arquivo de chave
client = vision.ImageAnnotatorClient()

# Obtém a chave de API do Google Generative AI do usuário
from google.colab import userdata
my_api_key = userdata.get("api_key")
genai.configure(api_key=my_api_key)

# Função para analisar a imagem
def analisar_imagem(content):
  image = vision.Image(content=content)

  # Inicializar a variável como lista vazia
  labels = []

  # Detecção de labels
  response = client.label_detection(image=image)
  if response.label_annotations:
      labels = [label.description for label in response.label_annotations if label.description is not None]

  return labels

# Obtém as preferências de decoração do usuário
preferencias = input("Descreva suas preferências de estilo de decoração: (ex: moderno, clássico, minimalista, rústico, etc.)")

# Solicita o upload da imagem do usuário
uploaded = files.upload()

# Extrai o conteúdo da imagem carregada
file_name = next(iter(uploaded))
content = uploaded[file_name]

# Analisa a imagem e extrai labels
labels = analisar_imagem(content)

# Define o modelo de linguagem a ser usado (text-bison-001 é bom para geração de texto)
model = "models/text-bison-001"

# Inicializar o tradutor
translator = Translator()

# Traduzir as preferências para inglês
preferencias_en = translator.translate(preferencias, dest='en').text

# Traduzir os labels para inglês
labels_en = [translator.translate(label, dest='en').text for label in labels if label]

# Traduzir a *prompt* para inglês
prompt_en = f"""
I have a room with the following elements: '{', '.join(labels_en)}'.
My current preferences are: '{preferencias_en}'.

Suggest specific changes I can make to the room's decor to achieve a different style,
considering the existing elements.
Focus on suggesting new styles that would complement the existing features.
"""

# Gerar sugestões de decoração em inglês
resposta = genai.generate_text(
    model='models/text-bison-001',
    prompt=prompt_en,
    temperature=0.7,
    max_output_tokens=200
)

# Traduz a resposta do modelo para português
resposta_pt = translator.translate(resposta.result, dest='pt').text

# Imprime as sugestões
print("\nSugestões:")
print(resposta_pt)



Descreva suas preferências de estilo de decoração: (ex: moderno, clássico, minimalista, rústico, etc.)clássico


Saving ambiente_rustico.jpg to ambiente_rustico.jpg

Sugestões:
*** Altere a paleta de cores. ** O estilo clássico é frequentemente caracterizado por cores neutras, como marrom, bege e branco.Se você deseja criar uma aparência diferente, tente adicionar cores mais brilhantes, como blues, verduras ou amarelos.Você também pode adicionar pops com travesseiros, lances ou obras de arte.
*** Adicione mais textura. ** O estilo clássico geralmente é bastante simplificado e minimalista.Para adicionar mais interesse, tente adicionar tecidos texturizados, como veludo, linho ou tweed.Você também pode adicionar tapetes, cestas ou outros objetos com texturas interessantes.
*** Mude os móveis. ** Os móveis clássicos geralmente são feitos de madeira e estofados em tecidos neutros.Se você deseja criar uma aparência diferente, tente usar móveis feitos de metal, vidro ou plástico.Você também pode escolher móveis com formas e desenhos mais interessantes.
*** Adicione mais plantas. ** As plantas são uma ót